In [2]:
import pandas as pd
import numpy as np
dir = 'd:\git\project1\data'

In [2]:
import torch
import pytorch_transformers
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [3]:
df = pd.read_csv(f'{dir}/tweets.csv', delimiter=';', nrows=100000)

In [5]:
df['tw'] = df['retweets'].apply(lambda x: 1 if x>0 else 0)

In [6]:
df.corr()['tw']

id         -0.060858
replies     0.135945
likes       0.174310
retweets    0.134363
tw          1.000000
Name: tw, dtype: float64

In [4]:
del df['replies']
del df['likes']
del df['user']
del df['id']
del df['fullname']
del df['url']
del df['timestamp']
df

,retweets,text
0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1,Another Test tweet that wasn't caught in the s...
3,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...
99995,0,"Never, ever touch bitcoin investments,bound to..."
99996,1,😂😂😂 facts #bitcoin https://t.co/mmI634HtKc
99997,0,"BitcoinLend Airdropping 1,000,000 BLEND ($1,00..."
99998,0,#inzura #tokensale\nNow is good time to get ri...


In [5]:
%%time
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

Wall time: 1min 16s


0        [101, 1041, 10439, 8189, 15529, 9956, 4895, 16...
1        [101, 4003, 6761, 1024, 15340, 4697, 12731, 14...
2        [101, 2178, 3231, 1056, 28394, 2102, 2008, 234...
3        [101, 2783, 19888, 2080, 7597, 999, 18411, 227...
4        [101, 11867, 12848, 1006, 16839, 2906, 8670, 2...
                               ...                        
99995    [101, 2196, 1010, 2412, 3543, 2978, 3597, 2378...
99996    [101, 100, 8866, 1001, 2978, 3597, 2378, 16770...
99997    [101, 2978, 3597, 2378, 7770, 2094, 2250, 2571...
99998    [101, 1001, 1999, 9759, 2527, 1001, 19204, 120...
99999    [101, 1060, 14536, 1010, 2978, 3597, 2378, 199...
Name: text, Length: 100000, dtype: object

In [6]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

307


In [7]:
tokenized.apply(lambda x: len(x)).sort_values()

65189      3
52340      3
65323      3
58469      3
80588      3
        ... 
33560    277
39104    278
72946    279
40999    305
41257    307
Name: text, Length: 100000, dtype: int64

In [8]:
tokens = pd.DataFrame()
tokens['retweets'] = df['retweets'].apply(lambda x: 1 if x>0 else 0)
max = 50
tokens['raw'] = tokenized.apply(lambda x: x if (len(x) < max) else None)
#tokens = tokens.dropna()
#tokens = tokenized.apply(token_lenght, max=30)

#tokens['raw'] = tokenized
tokens

,retweets,raw
0,0,"[101, 1041, 10439, 8189, 15529, 9956, 4895, 16..."
1,0,None
2,1,"[101, 2178, 3231, 1056, 28394, 2102, 2008, 234..."
3,0,None
4,0,None
...,...,...
99995,0,"[101, 2196, 1010, 2412, 3543, 2978, 3597, 2378..."
99996,1,"[101, 100, 8866, 1001, 2978, 3597, 2378, 16770..."
99997,0,None
99998,0,None


In [9]:
#tokens['retweets'].groupby('retweets').nunique()
tokens.groupby('retweets').count()

,raw
retweets,
0,41375
1,5907


In [14]:
%matplotlib
tokens['retweets'].plot.hist(bins=1000)

In [31]:
%%time
padded = np.array([i + [0]*(max-len(i)) if i!=None else [None]*(max) for i in tokens['raw']])
padded

Wall time: 992 ms


array([[101, 1041, 10439, ..., 0, 0, 0],
       [None, None, None, ..., None, None, None],
       [101, 2178, 3231, ..., 0, 0, 0],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [101, 1060, 14536, ..., 0, 0, 0]], dtype=object)

In [32]:
df_padded = pd.DataFrame(padded)
df_padded

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,101,1041,10439,8189,15529,9956,4895,16371,16059,2678,...,2015,102,0,0,0,0,0,0,0,0
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,101,2178,3231,1056,28394,2102,2008,2347,1005,1056,...,0,0,0,0,0,0,0,0,0,0
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,101,2196,1010,2412,3543,2978,3597,2378,10518,1010,...,1013,1050,4160,2575,8523,4160,5358,2497,102,0
99996,101,100,8866,1001,2978,3597,2378,16770,1024,1013,...,0,0,0,0,0,0,0,0,0,0
99997,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
99998,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [33]:
%%time
union = pd.concat([tokens['retweets'], df_padded], axis=1)
union.dropna(inplace=True)
union

Wall time: 355 ms


,retweets,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,101,1041,10439,8189,15529,9956,4895,16371,16059,...,2015,102,0,0,0,0,0,0,0,0
2,1,101,2178,3231,1056,28394,2102,2008,2347,1005,...,0,0,0,0,0,0,0,0,0,0
6,0,101,1030,22064,17139,23460,2057,2031,2042,2311,...,0,0,0,0,0,0,0,0,0,0
7,0,101,1030,7707,2532,12722,18609,15697,13820,2099,...,0,0,0,0,0,0,0,0,0,0
13,1,101,11241,2026,2166,10995,2046,2978,3597,2378,...,1013,29347,2102,16048,2102,2620,2050,26976,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99989,0,101,1030,19888,24163,29148,2129,2152,2978,3597,...,0,0,0,0,0,0,0,0,0,0
99992,0,101,1030,1996,2389,5558,1030,3682,2595,2099,...,0,0,0,0,0,0,0,0,0,0
99995,0,101,2196,1010,2412,3543,2978,3597,2378,10518,...,1013,1050,4160,2575,8523,4160,5358,2497,102,0
99996,1,101,100,8866,1001,2978,3597,2378,16770,1024,...,0,0,0,0,0,0,0,0,0,0


In [34]:
%%time
union.to_csv(f'{dir}/retweets_padded1.csv', index=False)

Wall time: 852 ms
